In [46]:
# ok I want with confidence, all zip codes reported by census, and their lat/lngs
import pandas as pd
import plotly.express as px
from IPython.display import display
pd.options.display.max_rows = 10

from lib import load_sweetened_zips

us_zips = load_sweetened_zips()
us_zips["_zip3"] = us_zips['_zip'].apply(lambda x: f"{x[0:3]}XX")
us_zips = us_zips.sort_values("_zip3").reset_index(drop=True)

df = us_zips.groupby(['_zip3']).agg(
    _lat=("_lat","mean"),
    _lng=("_lng","mean"),
    _census_total=("_census_total","sum")
).sort_values("_census_total", ascending=False).reset_index()
df



,_zip3,_lat,_lng,_census_total
0,770XX,29.776118,-95.415475,2906700
1,606XX,41.867574,-87.673702,2680484
2,112XX,40.652864,-73.955791,2504700
3,900XX,34.041931,-118.306209,2404395
4,945XX,37.940440,-122.056448,2274102
...,...,...,...,...
888,821XX,44.519971,-110.649091,369
889,205XX,38.892419,-77.029148,8
890,753XX,32.813992,-96.842808,0
891,204XX,38.897429,-77.047661,0


In [50]:
px.scatter_mapbox(
    df, lat="_lat", lon="_lng", size_max=15, mapbox_style="open-street-map", 
    height=600, zoom=3, 
    size="_census_total",
    title=f"Census Total | k={len(df)} 3-Digit Zips | n={df['_census_total'].sum()} "
).show()

In [59]:
df['_rare_total'] = df['_census_total'].apply(lambda x: int(x * 0.0006))
df = df[df['_rare_total'] > 0]

px.scatter_mapbox(
    df, lat="_lat", lon="_lng", size_max=15, mapbox_style="open-street-map", 
    height=600, zoom=3, 
    size="_census_total",
    title=f"Rare Total | k={len(df)} 3-Digit Zips | n={df['_rare_total'].sum()} "
).show()

/var/folders/vy/x5kxqd453qd7x76mzmstj7t80000gn/T/ipykernel_69331/2813118599.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
px.scatter(df, x='_lat', y='_lng', color='_rare_total')

In [72]:
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=10, init='k-means++',
    n_init=10, max_iter=300, 
)
y_km = km.fit_predict(df[['_lat','_lng']])
df['_y'] = y_km
df

px.scatter(df, x='_lat', y='_lng', color='_y')

In [77]:
df.groupby(['_y']).agg(rare_total=("_rare_total","sum"))

,rare_total
_y,
0,40214
1,8164
2,7752
3,24562
4,14077
5,30652
6,815
7,27200
8,33193
